In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import gc
from tqdm import tqdm

from pyspark.sql import SparkSession, functions as F 
from pyspark import StorageLevel
from pyspark.sql.types import *

In [ ]:
APP_NAME = "rs"

spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master("local[3]")\
    .config('spark.ui.port', "4040")\
    .config("spark.driver.memory", "2g")\
    .config("spark.executor.memory", "3g")\
    .getOrCreate()

spark

# Read 

In [4]:
domen = 'com'
cur_pd_books = pd.read_csv(f'new_{domen}_books_0_1056.csv').drop('Unnamed: 0', axis=1)

(154583, 9)


In [13]:
cur_items = pd.read_csv('new_book_ids.csv')

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType

schema = StructType([
    StructField("asin",LongType(),True), \
    StructField("reviewerID", StringType(),True), \
    StructField("reviewerName", StringType(), True), \
    StructField("overall", DoubleType(), True), \
    StructField("unixReviewTime", StringType(), True), \
    StructField("style", StringType(), True), \
    StructField("summary", StringType(), True), \
    StructField("reviewText", StringType(), True), \
    StructField("verified", StringType(), True)           
  ])

books_ = spark.createDataFrame(cur_pd_books)

# Processing

## asin

In [ ]:
# Delete beginning word 'book'. It was necessary for searching
books = books.withColumn('asin', F.regexp_replace('asin', r"^book ", ''))

## reviewerID

In [ ]:
# retrieve text between ''
books = books.withColumn('reviewerID', F.regexp_extract(books.reviewerID, r"\'([^\'\']+)\'", 1))

## overall

In [ ]:
# retriev text between '.
books = books.withColumn('overall', F.regexp_extract(books.rating, r"\'[^\'\']+\.", 0))
# delete extra characters '.
books = books.withColumn('overall', F.regexp_replace('overall', r"'|\.", ''))

## Title

In [ ]:
# retrieve text between ''
books = books.withColumn('summary', F.regexp_extract(books.summary, r"\'([^\'\']+)\'", 1))

## Body

description

In [ ]:
# Delete unnecesary beginning
books = books.withColumn('reviewText', F.regexp_replace('reviewText', r"\'(\\n|\s)*'", ''))
# retrieve text between []
books = books.withColumn('reviewText', F.regexp_extract(books.reviewText, r"\[([^\[\]]+)\]", 1))
# Delete signs: , '
books = books.withColumn('reviewText', F.regexp_replace('reviewText', r"(\',)|(, \')", ''))

## Date

In [ ]:
# retrieve text between ''
books = books.withColumn('unixReviewTime', F.regexp_extract(books.date, r"\'([^\'\']+)\'", 1))
books = books.select('unixReviewTime').persist(StorageLevle.DISK_ONLY)